### Лабораторная работа №5

#### <span style="color:blue">Задание.</span> Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Сначала мы импортируем необходимые библиотеки и нормализуем данные:

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

data = pd.read_csv("adult.data")
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']
data.columns = column_names

data = data.replace(' ?', pd.NA)

data = data.dropna()
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разделение данных на обучающий и тестовый наборы

In [11]:
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Метод решающего дерева

Создание модели решающего дерева и поиск лучших параметров.

In [12]:
decision_tree_model = DecisionTreeClassifier(random_state=42)

param_grid_decision_tree = {'max_depth': [5, 10, 15, 20]}

grid_search_decision_tree = GridSearchCV(decision_tree_model, param_grid_decision_tree, cv=5)

grid_search_decision_tree.fit(X_train, y_train)

print("Лучшие параметры решающего дерева: ", grid_search_decision_tree.best_params_)

Лучшие параметры решающего дерева:  {'max_depth': 10}


Обучение модели на обучающем наборе и оценка производительности модели на тестовом наборе.

In [13]:
best_decision_tree_model = grid_search_decision_tree.best_estimator_

best_decision_tree_model.fit(X_train, y_train)

y_pred_tree = best_decision_tree_model.predict(X_test)

accuracy_tree = accuracy_score(y_test, y_pred_tree)
report_tree = classification_report(y_test, y_pred_tree)

print(f'Точность решающего дерева: {accuracy_tree}')
print('Отчет о классификации решающего дерева:\n', report_tree)

Точность решающего дерева: 0.8448533068125311
Отчет о классификации решающего дерева:
               precision    recall  f1-score   support

           0       0.89      0.91      0.90      4535
           1       0.71      0.64      0.67      1498

    accuracy                           0.84      6033
   macro avg       0.80      0.78      0.79      6033
weighted avg       0.84      0.84      0.84      6033



#### Метод случайного леса

Создание модели случайного лесаи поиск лучших параметров.

In [14]:
random_forest_model = RandomForestClassifier(random_state=42)

param_grid_random_forest = {'n_estimators': [50, 100, 150], 'max_depth': [5, 10, 15, 20]}

grid_search_random_forest = GridSearchCV(random_forest_model, param_grid_random_forest, cv=5)

grid_search_random_forest.fit(X_train, y_train)

print("Лучшие параметры случайного леса: ", grid_search_random_forest.best_params_)

Лучшие параметры случайного леса:  {'max_depth': 20, 'n_estimators': 150}


Обучение модели на обучающем наборе и оценка производительности модели на тестовом наборе.

In [15]:
best_random_forest_model = grid_search_random_forest.best_estimator_

best_random_forest_model.fit(X_train, y_train)

y_pred_forest = best_random_forest_model.predict(X_test)

accuracy_forest = accuracy_score(y_test, y_pred_forest)
report_forest = classification_report(y_test, y_pred_forest)

print(f'Точность случайного леса: {accuracy_forest}')
print('Отчет о классификации случайного леса:\n', report_forest)

Точность случайного леса: 0.8571191778551301
Отчет о классификации случайного леса:
               precision    recall  f1-score   support

           0       0.88      0.94      0.91      4535
           1       0.77      0.61      0.68      1498

    accuracy                           0.86      6033
   macro avg       0.82      0.78      0.79      6033
weighted avg       0.85      0.86      0.85      6033



Для повышения надежности оценки производительности модели, проведем кросс-валидацию(например, с 5 фолдами) для решающего дерева и для случайного леса.

In [16]:
cv_scores_tree = cross_val_score(best_decision_tree_model, X, y, cv=5)
print('Результаты кросс-валидации решающего дерева:', cv_scores_tree)
print(f'Средняя точность решающего дерева: {cv_scores_tree.mean()}')

cv_scores_forest = cross_val_score(best_random_forest_model, X, y, cv=5)
print('Результаты кросс-валидации случайного леса:', cv_scores_forest)
print(f'Средняя точность случайного леса: {cv_scores_forest.mean()}')

Результаты кросс-валидации решающего дерева: [0.84485331 0.84930371 0.85278515 0.85576923 0.85593501]
Средняя точность решающего дерева: 0.8517292820521615
Результаты кросс-валидации случайного леса: [0.85314106 0.85875332 0.86190318 0.86339523 0.86240053]
Средняя точность случайного леса: 0.8599186624317798


Мы выбрали гиперпараметры (например, max_depth и n_estimators) на основе знаний о данных и провели оценку производительности моделей на тестовом наборе, а также использовали перекрестную проверку для оценки их обобщающей способности.